In [1]:
import torch 
import torch.nn as nn
import torchaudio
import numpy as np
from datasets import load_dataset, load_metric, Audio
import os
import torchaudio
import torch.nn.functional as F
from scipy import arange, around, array, linspace
from scipy.interpolate import interp1d
from scipy.signal import resample
from torchaudio import transforms

sr = 16000

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# hyper-parameters
TRAIN_ALL_WEIGHTS = True # すべてのパラメータを学習するか最後の全結合層だけ学習するか
lr = 1e-4 #学習率
num_train_epochs = 50 #学習するエポック数
per_device_train_batch_size = 8# GPU1枚あたりのバッチサイズ(32の約数)
torch.backends.cudnn.benchmark = True # 再現性がなくなるが高速化
tgt = 'hira' # ラベルの種類

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
torch.__version__

'1.10.2+cu113'

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import pandas as pd
train = pd.read_csv(f'./train_{tgt}_wav.csv', index_col=0)
val = pd.read_csv(f'./val_{tgt}_wav.csv', index_col=0)
train.head()

,path,sentence,up_votes,down_votes,age,gender,accents,locale,segment,hira
client_id,,,,,,,,,,
0161d251e9a710136aba6b9ed96572600657c1f79563b37fe36e77d84509789b879f4468eb74a199be08ea1665a62be94e61d61a47ace02acebfe396d5e34523,./data/processed_clips/common_voice_ja_2769521...,自己が物となること、自己がなくなることではない。,2,1,NaN,NaN,NaN,ja,NaN,じこがものとなることじこがなくなることではない
0161d251e9a710136aba6b9ed96572600657c1f79563b37fe36e77d84509789b879f4468eb74a199be08ea1665a62be94e61d61a47ace02acebfe396d5e34523,./data/processed_clips/common_voice_ja_2769521...,暗証番号をど忘れしてお金を引き出せない,3,0,NaN,NaN,NaN,ja,NaN,あんしょうばんごうをどわすれしておきんをひきだせない
025ba9df1f6603d5d87e104bcc48cd22ec3fafdf474c147158368920f5b156bab36fd42423c000f7e1ceee6467a8eaac47989ee0cb1c2d231399fd7fc8ec5635,./data/processed_clips/common_voice_ja_3068695...,それは唯心論でもなく神秘主義でもない。,2,1,NaN,NaN,NaN,ja,NaN,それはゆいしんろんでもなくしんぴしゅぎでもない
025ba9df1f6603d5d87e104bcc48cd22ec3fafdf474c147158368920f5b156bab36fd42423c000f7e1ceee6467a8eaac47989ee0cb1c2d231399fd7fc8ec5635,./data/processed_clips/common_voice_ja_3068695...,常識は先ず行為的知識である。,2,0,NaN,NaN,NaN,ja,NaN,じょうしきはまずこういてきちしきである
025ba9df1f6603d5d87e104bcc48cd22ec3fafdf474c147158368920f5b156bab36fd42423c000f7e1ceee6467a8eaac47989ee0cb1c2d231399fd7fc8ec5635,./data/processed_clips/common_voice_ja_3068695...,顔には何かあると書いてあるぞ？,2,0,NaN,NaN,NaN,ja,NaN,かおにはなにかあるとかいてあるぞ


In [7]:
val = val.reset_index()
train = train.reset_index()
len(train)

34215

In [8]:
from transformers import Wav2Vec2CTCTokenizer
tokenizer = Wav2Vec2CTCTokenizer(f"./vocab_{tgt}.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
tokenizer

PreTrainedTokenizer(name_or_path='', vocab_size=114, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'})

In [9]:
from transformers import Wav2Vec2FeatureExtractor
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=sr, padding_value=0.0, do_normalize=True, return_attention_mask=True)
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

In [10]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor

Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

- tokenizer: PreTrainedTokenizer(name_or_path='', vocab_size=114, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'})

In [11]:
from scipy import signal
def lpf(wave, fs, fe , n):
    nyq = fs / 2.0
    b, a = signal.butter(1, fe/nyq, btype='low')
    for i in range(0, n):
        wave = signal.filtfilt(b, a, wave)
    return wave

def hpf(wave, fs, fe , n):
    nyq = fs / 2.0
    b, a = signal.butter(1, fe/nyq, btype='high')
    for i in range(0, n):
        wave = signal.filtfilt(b, a, wave)
    return wave

In [13]:
stretch = torchaudio.transforms.TimeStretch()

In [14]:
class W2v2Dataset(torch.utils.data.Dataset):
    def __init__(self, df, train=False, p_wav=0.75, p_shf=0.75):
        self.df = df
        self.pathes = df['path'].values
        self.sentences = df[tgt].values
        self.train = train
        self.p_wav = p_wav
        self.p_shf = p_shf        

    def __getitem__(self, idx):
        waveform, sample_rate = torchaudio.load(self.pathes[idx], normalize=True)
        batch = dict()
        x = processor(waveform.reshape(-1), sampling_rate=sr).input_values[0]  
        orig_shape = x.shape
        
        if self.train:
            if torch.rand(1)>self.p_wav:
                x = torch.Tensor(x)
                rate = torch.rand(1).numpy()[0]*0.2 + 0.9 
                x = x.reshape(1,1,1,-1)
                x = F.interpolate(x, (1,int(rate*x.shape[-1])), mode='bilinear', align_corners=False)
                x = x.numpy()
                x = x.reshape(-1)

            if torch.rand(1)>self.p_wav:
                x = torch.Tensor(x)
                rate = torch.rand(1).numpy()[0]*0.2 + 0.9 
                transform = transforms.Resample(sample_rate, int(sample_rate*rate))
                x = transform(x)
                x = x.numpy()
                x = x.reshape(-1)

            if torch.rand(1)>self.p_wav:
                x += torch.normal(0, 1e-3, size=x.shape).numpy()

            if torch.rand(1)>self.p_wav:
                rate = torch.rand(1).numpy()[0]*2 + 7 
                n = torch.rand(1).numpy()[0]*8 + 3 
                x = lpf(x, sr, 2**rate, int(n//1))

            if torch.rand(1)>self.p_wav:
                rate = torch.rand(1).numpy()[0]*2 + 9 
                n = torch.rand(1).numpy()[0]*8 + 3 
                x = hpf(x, sr, 2**rate, int(n//1))

            k = 0
            while 1:
                if torch.rand(1)>self.p_shf or k>7: break
                mask_len = torch.randint(1, sr//100, (1,))[0].numpy()
                start = torch.randint(0, x.shape[0]-mask_len, (1,))[0].numpy()
                x[start:start+mask_len] = 0
                k += 1

            k = 0
            while 1:
                if torch.rand(1)>self.p_shf or k>7: break
                mask_len = torch.randint(1, sr//100, (1,))[0].numpy()
                start = torch.randint(0, x.shape[0]-mask_len, (1,))[0].numpy()
                rev = x[start:start+mask_len]
                rev = rev[::-1]
                x[start:start+mask_len] = rev
                k += 1
                
        batch["input_values"] = x
        
        with processor.as_target_processor():
            batch["labels"] = processor(self.sentences[idx]).input_ids       
        
        return batch

    def __len__(self):
        return len(self.df)

train_dataset = W2v2Dataset(train, train=True)
val_dataset = W2v2Dataset(val)

In [15]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = 512#None
    max_length_labels: Optional[int] = 512#None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [16]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
data_collator

DataCollatorCTCWithPadding(processor=Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

- tokenizer: PreTrainedTokenizer(name_or_path='', vocab_size=114, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}), padding=True, max_length=512, max_length_labels=512, pad_to_multiple_of=None, pad_to_multiple_of_labels=None)

In [17]:
# -*- coding: utf-8 -*-

#
# レーベンシュタイン距離を用いて，
# 認識結果の誤り数を算出します．
#

import numpy as np
import copy

def calculate_error(hypothesis, reference):
    ''' レーベンシュタイン距離を計算し，
        置換誤り，削除誤り，挿入誤りを出力する
    hypothesis:       認識結果(トークン毎に区切ったリスト形式)
    reference:        正解(同上)
    total_error:      総誤り数
    substitute_error: 置換誤り数
    delete_error:     削除誤り数
    insert_error:     挿入誤り数
    len_ref:          正解文のトークン数
    '''
    # 認識結果および正解系列の長さを取得
    len_hyp = len(hypothesis)
    len_ref = len(reference)

    # 累積コスト行列を作成する
    # 行列の各要素には，トータルコスト，
    # 置換コスト，削除コスト，挿入コストの
    # 累積値が辞書形式で定義される．
    cost_matrix = [[{"total":0, 
                     "substitute":0,
                     "delete":0,
                     "insert":0} \
                     for j in range(len_ref+1)] \
                         for i in range(len_hyp+1)]

    # 0列目と0行目の入力
    for i in range(1, len_hyp+1):
        # 縦方向への遷移は，削除処理を意味する
        cost_matrix[i][0]["delete"] = i
        cost_matrix[i][0]["total"] = i
    for j in range(1, len_ref+1):
        # 横方向への遷移は，挿入処理を意味する
        cost_matrix[0][j]["insert"] = j
        cost_matrix[0][j]["total"] = j

    # 1列目と1行目以降の累積コストを計算していく
    for i in range(1, len_hyp+1):
        for j in range(1, len_ref+1):
            #
            # 各処理のコストを計算する
            #
            # 斜め方向の遷移時，文字が一致しない場合は，
            # 置換処理により累積コストが1増加
            substitute_cost = \
                cost_matrix[i-1][j-1]["total"] \
                + (0 if hypothesis[i-1] == reference[j-1] else 1)
            # 縦方向の遷移時は，削除処理により累積コストが1増加
            delete_cost = cost_matrix[i-1][j]["total"] + 1
            # 横方向の遷移時は，挿入処理により累積コストが1増加
            insert_cost = cost_matrix[i][j-1]["total"] + 1

            # 置換処理，削除処理，挿入処理のうち，
            # どの処理を行えば累積コストが最も小さくなるかを計算
            cost = [substitute_cost, delete_cost, insert_cost]
            min_index = np.argmin(cost)

            if min_index == 0:
                # 置換処理が累積コスト最小となる場合

                # 遷移元の累積コスト情報をコピー
                cost_matrix[i][j] = \
                    copy.copy(cost_matrix[i-1][j-1])
                # 文字が一致しない場合は，
                # 累積置換コストを1増加させる
                cost_matrix[i][j]["substitute"] \
                    += (0 if hypothesis[i-1] \
                        == reference[j-1] else 1)
            elif min_index == 1:
                # 削除処理が累積コスト最小となる場合
                
                # 遷移元の累積コスト情報をコピー
                cost_matrix[i][j] = copy.copy(cost_matrix[i-1][j])
                # 累積削除コストを1増加させる
                cost_matrix[i][j]["delete"] += 1
            else:
                # 置換処理が累積コスト最小となる場合
                
                # 遷移元の累積コスト情報をコピー
                cost_matrix[i][j] = copy.copy(cost_matrix[i][j-1])
                # 累積挿入コストを1増加させる
                cost_matrix[i][j]["insert"] += 1

            # 累積トータルコスト(置換+削除+挿入コスト)を更新
            cost_matrix[i][j]["total"] = cost[min_index]

    #
    # エラーの数を出力する
    # このとき，削除コストは挿入誤り，
    # 挿入コストは削除誤りになる点に注意．
    # (削除コストが1である
    #    = 1文字削除しないと正解文にならない 
    #    = 認識結果は1文字分余計に挿入されている
    #    = 挿入誤りが1である)
    #

    # 累積コスト行列の右下の要素が最終的なコストとなる．
    total_error = cost_matrix[len_hyp][len_ref]["total"]
    substitute_error = cost_matrix[len_hyp][len_ref]["substitute"]
    # 削除誤り = 挿入コスト
    delete_error = cost_matrix[len_hyp][len_ref]["insert"]
    # 挿入誤り = 削除コスト
    insert_error = cost_matrix[len_hyp][len_ref]["delete"]
    
    # 各誤り数と，正解文の文字数
    # (誤り率を算出する際に分母として用いる)を出力
    return (total_error, 
            substitute_error,
            delete_error,
            insert_error,
            len_ref)

In [18]:
def mtr_leven(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    for i in range(5):print(pred_str[i],'\n',  label_str[i])
        
    # 各誤りの総数(エラー率算出時の分子)
    total_err = 0
    total_sub = 0
    total_del = 0
    total_ins = 0
    # 正解文の総文字数(エラー率算出時の分母)
    total_length = 0
    for i in range(len(pred_str)):
        (error, substitute, delete, insert, ref_length) \
                = calculate_error(pred_str[i], label_str[i])

        # 総誤り数を累積する
        total_err += error
        total_sub += substitute
        total_del += delete
        total_ins += insert
        total_length += ref_length
        
    err_rate = 100.0 * total_err / total_length
    sub_rate = 100.0 * total_sub / total_length
    del_rate = 100.0 * total_del / total_length
    ins_rate = 100.0 * total_ins / total_length

    return {'n_token': total_length, 'n_error': total_err, 'n_sub':total_sub, 'n_del':total_del, 'n_ins':total_ins,
            'ter': err_rate, 'r_sub': sub_rate, 'r_del': del_rate, 'r_ins': ins_rate}       

In [19]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    'facebook/wav2vec2-large-xlsr-53',
    attention_dropout=0.2,
    hidden_dropout=0.2,
    feat_proj_dropout=0.2,
    mask_time_prob=0.1,
    layerdrop=0.2,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    diversity_loss_weight=100
)

model.lm_head = nn.Linear(1024, len(processor.tokenizer))
model.config.vocab_size=len(processor.tokenizer)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'quantizer.codevectors', 'project_q.bias', 'project_hid.bias', 'project_hid.weight', 'quantizer.weight_proj.bias', 'quantizer.weight_proj.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to u

In [20]:
model.config

Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-large-xlsr-53",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.2,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 100,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.2,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0.0,
  "gradient_c

In [21]:
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elemen

In [22]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
pytorch_total_params

315555570

In [23]:
if TRAIN_ALL_WEIGHTS:
    for param in model.parameters():
        param.requires_grad = True
else:
    model.freeze_feature_extractor()

In [24]:
model.gradient_checkpointing_enable()

In [25]:
warmup_steps = 5 * len(train)//32
num_total_steps = num_train_epochs * len(train)//32

In [26]:
from transformers import TrainingArguments
training_args = TrainingArguments(
  output_dir=f"./wav2vec2_xlsr_ja_{tgt}",
  group_by_length=False,
  per_device_train_batch_size=per_device_train_batch_size,
  gradient_accumulation_steps=32//per_device_train_batch_size,
  per_device_eval_batch_size=per_device_train_batch_size,
  evaluation_strategy="epoch",
  num_train_epochs=num_train_epochs,
  fp16=False,
  save_strategy='epoch',
  fp16_full_eval=True,
  logging_steps=10,
  learning_rate=lr,
  warmup_steps=warmup_steps,
  save_total_limit=5,
  weight_decay=1e-5,
  dataloader_num_workers=4,
  prediction_loss_only=False,
  lr_scheduler_type='linear',
  seed=4,
  #eval_steps=10,
)
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=4,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=True,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=4,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
jit_mode_eval=False,
label_names=None,
label_smoothing_fact

In [27]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=mtr_leven,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=processor.feature_extractor
)
trainer

In [28]:
import gc
gc.collect()

0

In [29]:
%%time
trainer.train()

***** Running training *****
  Num examples = 34215
  Num Epochs = 50
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 53450


Epoch,Training Loss,Validation Loss,Er,Sr,Dr,Ir
0,4.149300,4.481318,100.000000,4.089776,95.910224,0.000000
1,4.077200,4.105532,100.000000,4.089776,95.910224,0.000000
2,3.594100,3.127913,98.004988,4.380715,93.615960,0.008313
3,2.056100,1.017686,26.716542,15.419784,2.967581,8.329177
4,1.746900,0.831252,22.593516,12.801330,2.302577,7.489609
5,1.537700,0.677111,20.689942,11.363259,1.961762,7.364921
6,1.592100,0.681609,20.315877,11.197007,1.670823,7.448047
7,1.590000,0.667006,19.152120,10.083126,1.413134,7.655860
8,1.491700,0.599228,18.869493,9.883624,1.404821,7.581047
9,1.472100,0.589231,17.988362,9.625935,1.396509,6.965919


***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


a 
 そぼはおおむねきげんよくさいころをころがしている
a 
 さいふをなくしたのでこうばんへいきます
a 
 せのたかさはいちしち〇せんちほどでめがおおきくややふとっている
a 
 あたらしいくつをはいてでかけます
a 
 かぞくといえどもことばでつたえるのはだいじ


Saving model checkpoint to ./wav2vec2_xlsr_ja_hira/checkpoint-1069
Configuration saved in ./wav2vec2_xlsr_ja_hira/checkpoint-1069/config.json
Model weights saved in ./wav2vec2_xlsr_ja_hira/checkpoint-1069/pytorch_model.bin
Feature extractor saved in ./wav2vec2_xlsr_ja_hira/checkpoint-1069/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


...

Saving model checkpoint to ./wav2vec2_xlsr_ja_hira/checkpoint-52381
Configuration saved in ./wav2vec2_xlsr_ja_hira/checkpoint-52381/config.json
Model weights saved in ./wav2vec2_xlsr_ja_hira/checkpoint-52381/pytorch_model.bin
Feature extractor saved in ./wav2vec2_xlsr_ja_hira/checkpoint-52381/preprocessor_config.json
Deleting older checkpoint [wav2vec2_xlsr_ja_hira/checkpoint-47036] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


そぼはおもにきねんおおくさいころころがしているa 
 そぼはおおむねきげんよくさいころをころがしている
さいふをなくしたのでこうばんへへいきますa 
 さいふをなくしたのでこうばんへいきます
せのたかさはひゃくらなじゅうせんちほどでめがおおきくややふとっているa 
 せのたかさはいちしち〇せんちほどでめがおおきくややふとっている
たらしいくつをはいっててかけますa 
 あたらしいくつをはいてでかけます
かぞくといえどもことばでつたえるのはだいじa 
 かぞくといえどもことばでつたえるのはだいじ


Saving model checkpoint to ./wav2vec2_xlsr_ja_hira/checkpoint-53450
Configuration saved in ./wav2vec2_xlsr_ja_hira/checkpoint-53450/config.json
Model weights saved in ./wav2vec2_xlsr_ja_hira/checkpoint-53450/pytorch_model.bin
Feature extractor saved in ./wav2vec2_xlsr_ja_hira/checkpoint-53450/preprocessor_config.json
Deleting older checkpoint [wav2vec2_xlsr_ja_hira/checkpoint-48105] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 1d 1h 51min 2s, sys: 2h 45min 4s, total: 1d 4h 36min 7s
Wall time: 3d 18h 19min 45s


TrainOutput(global_step=53450, training_loss=1.7673370755109081, metrics={'train_runtime': 325182.8395, 'train_samples_per_second': 5.261, 'train_steps_per_second': 0.164, 'total_flos': 3.8871539202226966e+20, 'train_loss': 1.7673370755109081, 'epoch': 50.0})